### ECS 289G Project
Emily Guo
Erjie Zhang

### Preprocess and Tokenization
Here, the multinews dataset as well as the tokenizer are loaded in. The preprocess function will tokenize the documents and summaries.

In [1]:
%pip install transformers datasets


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\EMMIE\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
from datasets import Dataset
from transformers import BartTokenizer
import json
import random

multinewsdata = 'mn_test_100.json'
with open(multinewsdata, 'r') as file:
    data = json.load(file)

random.shuffle(data)
train_split = int(0.7 * len(data))
dev_split = train_split + int(0.15 * len(data))
train_data = data[:train_split]
dev_data = data[train_split:dev_split]
test_data = data[dev_split:]

# Convert the list of dictionaries to the expected dictionary of lists format
def convert_to_dict_of_lists(data_list):
    return {key: [dic[key] for dic in data_list] for key in data_list[0]}

train_dict = convert_to_dict_of_lists(train_data)
dev_dict = convert_to_dict_of_lists(dev_data)
test_dict = convert_to_dict_of_lists(test_data)

# Initialize tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

# Function to preprocess and tokenize the data
def preprocess_function(examples):
    tokenized = tokenizer(examples["document"], max_length=1024, truncation=True, padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=256, truncation=True, padding='max_length')
    return {
        "input_ids": tokenized["input_ids"], 
        "attention_mask": tokenized["attention_mask"], 
        "labels": labels["input_ids"]
    }

# Create Hugging Face Dataset objects
train_dataset = Dataset.from_dict(train_dict)
dev_dataset = Dataset.from_dict(dev_dict)
test_dataset = Dataset.from_dict(test_dict)

# Apply the preprocessing and tokenization
train_dataset = train_dataset.map(preprocess_function, batched=True)
dev_dataset = dev_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# The datasets are now ready to be used for training and evaluation


Map:   0%|          | 0/70 [00:00<?, ? examples/s]c:\Users\EMMIE\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 15/15 [00:00<00:00, 56.12 examples/s]
